In [1]:
import numpy as np
from copy import copy, deepcopy
import queue
from collections import OrderedDict
import time
import random
%run utilityFunc.ipynb
%run Node.ipynb
%run HeuristicFuntions.ipynb
%run CustomizedPriotiryQueue.ipynb

0
0
4


In [2]:
def get_successor(puzzle_dim_shape, elt_position, puzzle, cost):
    original_puzzle = deepcopy(puzzle)
    successor = []
    successor.append((regular_move(puzzle_dim_shape, elt_position, puzzle, "up"), original_puzzle, cost+1))
    successor.append((regular_move(puzzle_dim_shape, elt_position, puzzle, "down"), original_puzzle, cost+1))
    successor.append((regular_move(puzzle_dim_shape, elt_position, puzzle, "left"), original_puzzle, cost+1))
    successor.append((regular_move(puzzle_dim_shape, elt_position, puzzle, "right"), original_puzzle, cost+1))
    successor.append((wrapping_move(puzzle_dim_shape, elt_position, puzzle), original_puzzle, cost+2))
    successor.append((opposed_diagonally_move(puzzle_dim_shape, elt_position, puzzle), original_puzzle, cost+3))
    successor.append((adjacent_diagonally_move(puzzle_dim_shape, elt_position, puzzle), original_puzzle, cost+3))

    successor = [i for i in successor if i[0] is not None] # eliminate None puzzle node
    return successor


# return a list including all goal state
def generate_goal(puzzle):
    goal = []
    first_dim, second_dim = puzzle.shape
    puzzle_flatten = puzzle.flatten() # flatten to one dimentinal
    sorted_puzzle = np.sort(puzzle_flatten) # sort array
    sorted_puzzle = np.append(sorted_puzzle[1:], 0)

    goal1 = sorted_puzzle.reshape(first_dim, second_dim)
    goal.append(goal1)
    
    goal2 = np.full((first_dim, second_dim), 0)
    idx = 0
    for s in range(second_dim):
        for f in range(first_dim):
            goal2[f, s] = sorted_puzzle[idx]
            idx += 1
    goal.append(goal2)
    # print(goal)
    return goal


def check_goal(node, goal):
    current_node = node.get_current_puzzle()
    return (current_node == goal[0]).all() or (current_node == goal[1]).all()

# def check_goal(node, goal):
#     current_node = node.get_current_puzzle()
#     return (current_node == goal).all()

def is_exist_in_closedList(node, closed_list):
    isExist = False
    for curr_node in closed_list:
        isExist = isExist or curr_node.__eq__(node)
    return isExist

def removeOrIgnore(node, closed_list, openQueue):
    index = 0
    for i in range(len(closed_list)):
        if closed_list[i].__eq__(node):
            index = i
    if closed_list[index].get_f_val() >= node.get_f_val():
        closed_list.remove(closed_list[index])
        openQueue.put(node)
    return
    
    


# get_successor(puzzle_dim_shape, sol, puzzle_test, 0)
# goal = generate_goal(puzzle_test)
# puzzle_test = deepcopy(goal[1])
# print(check_goal(puzzle_test, goal))



In [3]:
#Uniform Cost Algo
def uniform_cost_algo(start_puzzle, goal):
    start_time = time.time()
    total_cost = 0
    closed_list = list() # elt: (Node(node, parent_node, h_val, g_val))
    open_queue = CustomizedPriotiryQueue("uniform_cost") # list: [Node(node, parent_node, h_val, g_val), ...]
    open_queue.put(Node(start_puzzle, None, 0, total_cost))
    
    while open_queue:
        if time.time() - start_time >= 60.0:
            return None
        node = open_queue.get()
        #print(node)
        puzzle = node.get_current_puzzle()
        total_cost = node.get_g_val()

        if not is_exist_in_closedList(node, closed_list):
            closed_list.append(node)
            
        if check_goal(node, goal):
            return closed_list
            
        elt_position = np.argwhere(puzzle == 0)
        successor = get_successor(puzzle_dim_shape, elt_position, puzzle, total_cost)
        for node_tuple in successor:
            successor_node = Node(node_tuple[0], node, 0, node_tuple[2])
            if not is_exist_in_closedList(successor_node, closed_list):
                open_queue.put(successor_node)
    return None

In [4]:
# greedy best first(GBFS) algo
def GBFS_algo(start_puzzle, goal_puzzle, h_type):
    start_time = time.time()
    total_cost = 0
    closed_list = list() # elt: (Node(node, parent_node, h_val, g_val))
    open_queue = CustomizedPriotiryQueue("GBFS")

    #calculate heuristic for start puzzle
    if h_type == 0:
        h = simple_heuristic(start_puzzle, goal_puzzle)
    if h_type == 1:
        h = enhanced_Manhattan(start_puzzle, goal_puzzle)
    if h_type == 2:
        h = rowColumnEvaluation(start_puzzle, goal_puzzle)
    open_queue.put(Node(start_puzzle, None, h, total_cost))
    
    while open_queue:
        if time.time() - start_time >= 60.0:
            return None
        node = open_queue.get()
        puzzle = node.get_current_puzzle()
        total_cost = node.get_g_val()

        if not is_exist_in_closedList(node, closed_list):
            closed_list.append(node)
            
        if check_goal(node, goal):
            return closed_list
            
        elt_position = np.argwhere(puzzle == 0)
        successor = get_successor(puzzle_dim_shape, elt_position, puzzle, total_cost)
        for node_tuple in successor:
            if h_type == 0:
                h = simple_heuristic(node_tuple[0], goal_puzzle)
            if h_type == 1:
                h = enhanced_Manhattan(node_tuple[0], goal_puzzle)
            if h_type == 2:
                h = rowColumnEvaluation(node_tuple[0], goal_puzzle)
            successor_node = Node(node_tuple[0], node, h, node_tuple[2])
            if not is_exist_in_closedList(successor_node, closed_list):
                open_queue.put(successor_node)
    return None

In [5]:
# A* algo
def aStarAlgo(start_puzzle, goal_puzzle, heuristic):
    total_cost = 0
    closed_list = list() # elt: (Node(node, parent_node, h_val, g_val))
    open_queue = CustomizedPriotiryQueue("aStar")
    start_time = time.time()
    
    #calculate heuristic for start puzzle
    if heuristic == 0:
        h = simple_heuristic(start_puzzle, goal_puzzle)
    if heuristic == 1:
        h = enhanced_Manhattan(start_puzzle, goal_puzzle)
    if heuristic == 2:
        h = rowColumnEvaluation(start_puzzle, goal_puzzle)
    open_queue.put(Node(start_puzzle, None, h, total_cost))
    
    while open_queue:
        if time.time() - start_time >= 60.0:
            return None
        node = open_queue.get()
        puzzle = node.get_current_puzzle()
        total_cost = node.get_g_val()

        if not is_exist_in_closedList(node, closed_list):
            closed_list.append(node)
            
        if check_goal(node, goal_puzzle):
            return closed_list
            
        elt_position = np.argwhere(puzzle == 0)
        successor = get_successor(puzzle_dim_shape, elt_position, puzzle, total_cost)
        for node_tuple in successor:
            if heuristic == 0:
                h = simple_heuristic(node_tuple[0], goal_puzzle)
            if heuristic == 1:
                h = enhanced_Manhattan(node_tuple[0], goal_puzzle)
            if heuristic == 2:
                h = rowColumnEvaluation(node_tuple[0], goal_puzzle) 
            successor_node = Node(node_tuple[0], node, h, node_tuple[2])
            if not is_exist_in_closedList(successor_node, closed_list):
                open_queue.put(successor_node)
            else:
                removeOrIgnore(successor_node,closed_list,open_queue)
    return None

In [6]:
# Generating a file with 50 puzzles
for k in range(50):
    line_list = list()
    for i in range(8):
        temp = random.randint(0,7)
        while temp in line_list:
            temp = random.randint(0,7)
        line_list.append(temp)
    # write to the file
    f = open("puzzle.txt", "a")
    for i in range(len(line_list)):
        b = str(line_list[i])
        if i != len(line_list) -1:
            f.write(b + " ")
        else:
            f.write(b + "\n")
f.close()

In [7]:
# reading the file
input = open("puzzle.txt", "r")
puzzle_list = [] # contains list of nparray for each puzzle
for line in input:
    puzzle_nparr = np.array(list(line.split(" ")), dtype=int)
    dim1_len = int(len(puzzle_nparr)/4) #calculate length of first dimension
    puzzle_nparr = puzzle_nparr.reshape(dim1_len, 4) #reshape nparray to 2*4 multi-dimentional array
    puzzle_list.append(puzzle_nparr)



In [8]:
# outputing search path & solution path
puzzle_dim_shape = puzzle_list[0].shape
for k in range(50):
    
    goal = generate_goal(puzzle_list[k])
    # outputing search files
    uniform_cost_search_time = 0.0
    # uniform cost search
    start_time = time.time()
    uniform_cost_search_path = uniform_cost_algo(puzzle_list[k],goal)
    end_time = time.time()
    uniform_cost_search_time = end_time-start_time
    f = open(str(k) + "_ucs_search.txt", "w")
    if uniform_cost_search_path == None:
        f.write("No solution")
    else: 
        for i in range(len(uniform_cost_search_path)):
            f.write(str(uniform_cost_search_path[i].get_f_val()) + " " + str(uniform_cost_search_path[i].get_g_val())+ " " + str(uniform_cost_search_path[i].get_h_val()))
            temp = uniform_cost_search_path[i].get_current_puzzle()
            x,y = temp.shape
            for m in range(x):
                for n in range(y):
                    f.write(" " + str(temp[m][n]))
            f.write("\n") 
    f.close()
    # greedy best first search
    # h1
    GBFS_h1_search_time = 0.0
    start_time = time.time()
    GBFS_search_path_h1 = GBFS_algo(puzzle_list[k], goal,1)
    end_time = time.time()
    GBFS_h1_search_time = end_time - start_time
    f = open(str(k) + "_gbfs-h1_search.txt", "w")
    if GBFS_search_path_h1 == None:
        f.write("No solution")
    else:
        for i in range(len(GBFS_search_path_h1)):
            f.write(str(GBFS_search_path_h1[i].get_f_val()) + " " + str(GBFS_search_path_h1[i].get_g_val())+ " " + str(GBFS_search_path_h1[i].get_h_val()))
            temp = GBFS_search_path_h1[i].get_current_puzzle()
            x,y = temp.shape
            for m in range(x):
                for n in range(y):
                    f.write(" " + str(temp[m][n]))
            f.write("\n") 
    f.close()
    # h2
    GBFS_h2_search_time = 0.0
    start_time = time.time()
    GBFS_search_path_h2 = GBFS_algo(puzzle_list[k], goal,2)
    end_time = time.time()
    GBFS_h2_search_time = end_time - start_time
    f = open(str(k) + "_gbfs-h2_search.txt", "w")
    if GBFS_search_path_h2 == None:
        f.write("No Solution")
    else:
        for i in range(len(GBFS_search_path_h2)):
            f.write(str(GBFS_search_path_h2[i].get_f_val()) + " " + str(GBFS_search_path_h2[i].get_g_val())+ " " + str(GBFS_search_path_h2[i].get_h_val()))
            temp = GBFS_search_path_h2[i].get_current_puzzle()
            x,y = temp.shape
            for m in range(x):
                for n in range(y):
                    f.write(" " + str(temp[m][n]))
            f.write("\n") 
    f.close()
    # A* 
    # h1
    astar_h1_search_time = 0.0
    start_time = time.time()
    a_star_search_path_h1 = aStarAlgo(puzzle_list[k], goal, 1)
    end_time = time.time()
    astar_h1_search_time = end_time - start_time
    f = open(str(k) + "_astar-h1_search.txt", "w")
    if a_star_search_path_h1 == None:
        f.write("No solution")
    else:
        for i in range(len(a_star_search_path_h1)):
            f.write(str(a_star_search_path_h1[i].get_f_val()) + " " + str(a_star_search_path_h1[i].get_g_val())+ " " + str(a_star_search_path_h1[i].get_h_val()))
            temp = a_star_search_path_h1[i].get_current_puzzle()
            x,y = temp.shape
            for m in range(x):
                for n in range(y):
                    f.write(" " + str(temp[m][n]))
            f.write("\n") 
    f.close()
    # h2
    astar_h2_search_time = 0.0
    start_time = time.time()
    a_star_search_path_h2 = aStarAlgo(puzzle_list[k], goal, 2)
    end_time = time.time()
    astar_h2_search_time = end_time - start_time
    f = open(str(k) + "_astar-h2_search.txt", "w")
    if a_star_search_path_h2 == None:
        f.write("No solution")
    else:
        for i in range(len(a_star_search_path_h2)):
            f.write(str(a_star_search_path_h2[i].get_f_val()) + " " + str(a_star_search_path_h2[i].get_g_val())+ " " + str(a_star_search_path_h2[i].get_h_val()))
            temp = a_star_search_path_h2[i].get_current_puzzle()
            x,y = temp.shape
            for m in range(x):
                for n in range(y):
                    f.write(" " + str(temp[m][n]))
            f.write("\n") 
    f.close()
    # writing solution files
    # uniform cost search
    f = open(str(k) + "_ucs_solution.txt", "w")
    solution_list = []
    if uniform_cost_search_path == None:
        f.write("No solution")
    else:
        # Finding the solution path
        temp3 = uniform_cost_search_path[len(uniform_cost_search_path)-1]
        temp4 = temp3.get_parent_puzzle()
        solution_list.append(temp3)
        while temp4 is not None:
            temp3 = temp4
            temp4 = temp3.get_parent_puzzle()
            solution_list.append(temp3)
        for j in range(len(solution_list)-1, -1, -1):
            if j == len(solution_list)-1:
                f.write("0 0")
                temp = solution_list[j].get_current_puzzle()
                x,y = temp.shape
                for m in range(x):
                    for n in range(y):
                        f.write(" " + str(temp[m][n]))
                f.write("\n")
            else:
                # finding which tile was moved
                temp1 = solution_list[j].get_current_puzzle()
                temp2 = solution_list[j+1].get_current_puzzle()
                x,y = temp1.shape
                tile = 0
                for m in range(x):
                    for n in range(y):
                        if temp1[m][n]!=temp2[m][n] and temp1[m][n] != 0:
                            tile = temp1[m][n]
                cost = solution_list[j].get_g_val() - solution_list[j+1].get_g_val()
                f.write(str(tile) + " " + str(cost))
                for m in range(x):
                    for n in range(y):
                        f.write(" " + str(temp1[m][n]))
                f.write("\n")
        f.write(str(solution_list[0].get_g_val())+" ")
        f.write(str(round(uniform_cost_search_time,2)))
    f.close()
    # greedy best first search
    # h1
    f = open(str(k) + "_gbfs-h1_solution.txt", "w")
    solution_list = []
    if GBFS_search_path_h1 == None:
        f.write("No solution")
    else:
        # Finding the solution path
        temp3 = GBFS_search_path_h1[len(GBFS_search_path_h1)-1]
        temp4 = temp3.get_parent_puzzle()
        solution_list.append(temp3)
        while temp4 is not None:
            temp3 = temp4
            temp4 = temp3.get_parent_puzzle()
            solution_list.append(temp3)
        for j in range(len(solution_list)-1, -1, -1):
            if j == len(solution_list)-1:
                f.write("0 0")
                temp = solution_list[j].get_current_puzzle()
                x,y = temp.shape
            
                for m in range(x):
                    for n in range(y):
                        f.write(" " + str(temp[m][n]))
                f.write("\n")
            else:
                # finding which tile was moved
                temp1 = solution_list[j].get_current_puzzle()
                temp2 = solution_list[j+1].get_current_puzzle()
                x,y = temp1.shape
                tile = 0
                for m in range(x):
                    for n in range(y):
                        if temp1[m][n]!=temp2[m][n] and temp1[m][n] != 0:
                            tile = temp1[m][n]
                cost = solution_list[j].get_g_val() - solution_list[j+1].get_g_val()
                f.write(str(tile) + " " + str(cost))
                for m in range(x):
                    for n in range(y):
                        f.write(" " + str(temp1[m][n]))
                f.write("\n")
        f.write(str(solution_list[0].get_g_val())+" ")
        f.write(str(round(GBFS_h1_search_time,2)))
    f.close()
    # h2 
    f = open(str(k) + "_gbfs-h2_solution.txt", "w")
    solution_list = []
    if GBFS_search_path_h2 == None:
        f.write("No solution")
    else:
        # Finding the solution path
        temp3 = GBFS_search_path_h2[len(GBFS_search_path_h2)-1]
        temp4 = temp3.get_parent_puzzle()
        solution_list.append(temp3)
        while temp4 is not None:
            temp3 = temp4
            temp4 = temp3.get_parent_puzzle()
            solution_list.append(temp3)
        for j in range(len(solution_list)-1, -1, -1):
            if j == len(solution_list)-1:
                f.write("0 0")
                temp = solution_list[j].get_current_puzzle()
                x,y = temp.shape
            
                for m in range(x):
                    for n in range(y):
                        f.write(" " + str(temp[m][n]))
                f.write("\n")
            else:
                # finding which tile was moved
                temp1 = solution_list[j].get_current_puzzle()
                temp2 = solution_list[j+1].get_current_puzzle()
                x,y = temp1.shape
                tile = 0
                for m in range(x):
                    for n in range(y):
                        if temp1[m][n]!=temp2[m][n] and temp1[m][n] != 0:
                            tile = temp1[m][n]
                cost = solution_list[j].get_g_val() - solution_list[j+1].get_g_val()
                f.write(str(tile) + " " + str(cost))
                for m in range(x):
                    for n in range(y):
                        f.write(" " + str(temp1[m][n]))
                f.write("\n")
        f.write(str(solution_list[0].get_g_val())+" ")
        f.write(str(round(GBFS_h2_search_time,2)))
    f.close()
    # A*
    # h1
    f = open(str(k) + "_astar-h1_solution.txt", "w")
    solution_list = []
    if a_star_search_path_h1 == None:
        f.write("No solution")
    else:
        # Finding the solution path
        temp3 = a_star_search_path_h1[len(a_star_search_path_h1)-1]
        temp4 = temp3.get_parent_puzzle()
        solution_list.append(temp3)
        while temp4 is not None:
            temp3 = temp4
            temp4 = temp3.get_parent_puzzle()
            solution_list.append(temp3)
        for j in range(len(solution_list)-1, -1, -1):
            if j == len(solution_list)-1:
                f.write("0 0")
                temp = solution_list[j].get_current_puzzle()
                x,y = temp.shape
            
                for m in range(x):
                    for n in range(y):
                        f.write(" " + str(temp[m][n]))
                f.write("\n")
            else:
                # finding which tile was moved
                temp1 = solution_list[j].get_current_puzzle()
                temp2 = solution_list[j+1].get_current_puzzle()
                x,y = temp1.shape
                tile = 0
                for m in range(x):
                    for n in range(y):
                        if temp1[m][n]!=temp2[m][n] and temp1[m][n] != 0:
                            tile = temp1[m][n]
                cost = solution_list[j].get_g_val() - solution_list[j+1].get_g_val()
                f.write(str(tile) + " " + str(cost))
                for m in range(x):
                    for n in range(y):
                        f.write(" " + str(temp1[m][n]))
                f.write("\n")
        f.write(str(solution_list[0].get_g_val()) + " ")
        f.write(str(round(astar_h1_search_time,2)))
    f.close()
    # h2
    f = open(str(k) + "_astar-h2_solution.txt", "w")
    solution_list = []
    if a_star_search_path_h2 == None:
        f.write("No solution")
    else:
        # Finding the solution path
        temp3 = a_star_search_path_h2[len(a_star_search_path_h2)-1]
        temp4 = temp3.get_parent_puzzle()
        solution_list.append(temp3)
        while temp4 is not None:
            temp3 = temp4
            temp4 = temp3.get_parent_puzzle()
            solution_list.append(temp3)
        for j in range(len(solution_list)-1, -1, -1):
            if j == len(solution_list)-1:
                f.write("0 0")
                temp = solution_list[j].get_current_puzzle()
                x,y = temp.shape
                
                for m in range(x):
                    for n in range(y):
                        f.write(" " + str(temp[m][n]))
                f.write("\n")
            else:
                # finding which tile was moved
                temp1 = solution_list[j].get_current_puzzle()
                temp2 = solution_list[j+1].get_current_puzzle()
                x,y = temp1.shape
                tile = 0
                for m in range(x):
                    for n in range(y):
                        if temp1[m][n]!=temp2[m][n] and temp1[m][n] != 0:
                            tile = temp1[m][n]
                cost = solution_list[j].get_g_val() - solution_list[j+1].get_g_val()
                f.write(str(tile) + " " + str(cost))
                for m in range(x):
                    for n in range(y):
                        f.write(" " + str(temp1[m][n]))
                f.write("\n")
        f.write(str(solution_list[0].get_g_val()) + " ")
        f.write(str(round(astar_h2_search_time,2)))
    f.close()
    
        
    

In [55]:
# total & average length of search and solution path
print("Uniform Cose Search:")
# uniform cost search
total_search_path_length = 0
total_solution_path_length = 0
total_no_solution_search = 0
total_no_solution_solution = 0
for i in range(50):
    input = open(str(i)+"_ucs_search.txt", "r")
    for lines in input:
        if lines == "No solution":
            total_no_solution_search += 1
        else:
            total_search_path_length += 1
            
print("no solution in search files: " + str(total_no_solution_search))
print("total length of search path: " + str(total_search_path_length))
average = total_search_path_length/(50-total_no_solution_search)
print("Average length of search path: " + str(round(average,2)))

for i in range(50):
    input = open(str(i)+"_ucs_solution.txt", "r")
    for lines in input:
        if lines == "No solution":
            total_no_solution_solution += 1
        else:
            if len(lines) >= 4:
                total_solution_path_length += 1
print("no solution in solution files: " + str(total_no_solution_solution))
print("total length of solution path: " + str(total_solution_path_length))
average = total_solution_path_length/(50-total_no_solution_solution)
print("Average length of solution path: " + str(round(average,2)))
print("Percentage of no solutions: " + str(round((total_no_solution_solution/50),2)))
                

Uniform Cose Search:
no solution in search files: 47
total length of search path: 4438
Average length of search path: 1479.33
no solution in solution files: 47
total length of solution path: 32
Average length of solution path: 10.67
Percentage of no solutions: 0.94


In [56]:
# total & average length of search and solution path
print("GBFS-h1:")
total_search_path_length = 0
total_solution_path_length = 0
total_no_solution_search = 0
total_no_solution_solution = 0
for i in range(50):
    input = open(str(i)+"_gbfs-h1_search.txt", "r")
    for lines in input:
        if lines == "No solution":
            total_no_solution_search += 1
        else:
            total_search_path_length += 1
            
print("no solution in search files: " + str(total_no_solution_search))
print("total length of search path: " + str(total_search_path_length))
average = total_search_path_length/(50-total_no_solution_search)
print("Average length of search path: " + str(round(average,2)))

for i in range(50):
    input = open(str(i)+"_gbfs-h1_solution.txt", "r")
    for lines in input:
        if lines == "No solution":
            total_no_solution_solution += 1
        else:
            if len(lines) >= 4:
                total_solution_path_length += 1
print("no solution in solution files: " + str(total_no_solution_solution))
print("total length of solution path: " + str(total_solution_path_length))
average = total_solution_path_length/(50-total_no_solution_solution)
print("Average length of solution path: " + str(round(average,2)))
print("Percentage of no solutions: " + str(round((total_no_solution_solution/50),2)))

GBFS-h1:
no solution in search files: 0
total length of search path: 2701
Average length of search path: 54.02
no solution in solution files: 0
total length of solution path: 819
Average length of solution path: 16.38
Percentage of no solutions: 0.0


In [58]:
# total & average length of search and solution path
print("GBFS-h2:")
total_search_path_length = 0
total_solution_path_length = 0
total_no_solution_search = 0
total_no_solution_solution = 0
for i in range(50):
    input = open(str(i)+"_gbfs-h2_search.txt", "r")
    for lines in input:
        if lines == "No solution":
            total_no_solution_search += 1
        else:
            total_search_path_length += 1
            
print("no solution in search files: " + str(total_no_solution_search))
print("total length of search path: " + str(total_search_path_length))
average = total_search_path_length/(50-total_no_solution_search)
print("Average length of search path: " + str(round(average,2)))

for i in range(50):
    input = open(str(i)+"_gbfs-h2_solution.txt", "r")
    for lines in input:
        if lines == "No solution":
            total_no_solution_solution += 1
        else:
            if len(lines) >= 4:
                total_solution_path_length += 1
print("no solution in solution files: " + str(total_no_solution_solution))
print("total length of solution path: " + str(total_solution_path_length))
average = total_solution_path_length/(50-total_no_solution_solution)
print("Average length of solution path: " + str(round(average,2)))
print("Percentage of no solutions: " + str(round((total_no_solution_solution/50),2)))

GBFS-h2:
no solution in search files: 0
total length of search path: 2806
Average length of search path: 56.12
no solution in solution files: 0
total length of solution path: 835
Average length of solution path: 16.7
Percentage of no solutions: 0.0


In [59]:
# total & average length of search and solution path
print("astar-h1:")
total_search_path_length = 0
total_solution_path_length = 0
total_no_solution_search = 0
total_no_solution_solution = 0
for i in range(50):
    input = open(str(i)+"_astar-h1_search.txt", "r")
    for lines in input:
        if lines == "No solution":
            total_no_solution_search += 1
        else:
            total_search_path_length += 1
            
print("no solution in search files: " + str(total_no_solution_search))
print("total length of search path: " + str(total_search_path_length))
average = total_search_path_length/(50-total_no_solution_search)
print("Average length of search path: " + str(round(average,2)))

for i in range(50):
    input = open(str(i)+"_astar-h1_solution.txt", "r")
    for lines in input:
        if lines == "No solution":
            total_no_solution_solution += 1
        else:
            if len(lines) >= 4:
                total_solution_path_length += 1
print("no solution in solution files: " + str(total_no_solution_solution))
print("total length of solution path: " + str(total_solution_path_length))
average = total_solution_path_length/(50-total_no_solution_solution)
print("Average length of solution path: " + str(round(average,2)))
print("Percentage of no solutions: " + str(round((total_no_solution_solution/50),2)))

astar-h1:
no solution in search files: 0
total length of search path: 8605
Average length of search path: 172.1
no solution in solution files: 0
total length of solution path: 623
Average length of solution path: 12.46
Percentage of no solutions: 0.0


In [61]:
# total & average length of search and solution path
print("astar-h2:")
total_search_path_length = 0
total_solution_path_length = 0
total_no_solution_search = 0
total_no_solution_solution = 0
for i in range(50):
    input = open(str(i)+"_astar-h2_search.txt", "r")
    for lines in input:
        if lines == "No solution":
            total_no_solution_search += 1
        else:
            total_search_path_length += 1
            
print("no solution in search files: " + str(total_no_solution_search))
print("total length of search path: " + str(total_search_path_length))
average = total_search_path_length/(50-total_no_solution_search)
print("Average length of search path: " + str(round(average,2)))

for i in range(50):
    input = open(str(i)+"_astar-h2_solution.txt", "r")
    for lines in input:
        if lines == "No solution":
            total_no_solution_solution += 1
        else:
            if len(lines) >= 4:
                total_solution_path_length += 1
print("no solution in solution files: " + str(total_no_solution_solution))
print("total length of solution path: " + str(total_solution_path_length))
average = total_solution_path_length/(50-total_no_solution_solution)
print("Average length of solution path: " + str(round(average,2)))
print("Percentage of no solutions: " + str(round((total_no_solution_solution/50),2)))

astar-h2:
no solution in search files: 0
total length of search path: 23509
Average length of search path: 470.18
no solution in solution files: 0
total length of solution path: 619
Average length of solution path: 12.38
Percentage of no solutions: 0.0


In [84]:
# average and total cost & execution time
print("Uniform cost search:")
total_cost = 0
total_execution_time = 0.0
total_no_solution = 0
for i in range(50):
    line_list = []
    input = open(str(i)+"_ucs_solution.txt","r")
    for lines in input:
        if lines == "No solution":
            total_execution_time += 60.0
            total_no_solution += 1
        else:
            line_list = input.readlines()
    if len(line_list) == 0:
        a = 0
    else:
        temp = line_list[len(line_list)-1]
        temp1 = temp.split()
        total_cost += float(temp1[0])
        total_execution_time += float(temp1[1])

        


print("total cost: " + str(total_cost))
print("average cost: " + str(round((total_cost/(50-total_no_solution)),2)))
print("total execution time: " + str(round(total_execution_time,2)))
print("Average execution time: " + str(round(total_execution_time/50,2)))
        
    
    


Uniform cost search:
total cost: 28.0
average cost: 9.33
total execution time: 2887.44
Average execution time: 57.75


In [85]:
print("GBFS-h1:")
total_cost = 0
total_execution_time = 0.0
total_no_solution = 0
for i in range(50):
    line_list = []
    input = open(str(i)+"_gbfs-h1_solution.txt","r")
    for lines in input:
        if lines == "No solution":
            total_execution_time += 60.0
            total_no_solution += 1
        else:
            line_list = input.readlines()
    if len(line_list) == 0:
        a = 0
    else:
        temp = line_list[len(line_list)-1]
        temp1 = temp.split()
        total_cost += float(temp1[0])
        total_execution_time += float(temp1[1])

        


print("total cost: " + str(total_cost))
print("average cost: " + str(round((total_cost/(50-total_no_solution)),2)))
print("total execution time: " + str(round(total_execution_time,2)))
print("Average execution time: " + str(round(total_execution_time/50,2)))

GBFS-h1:
total cost: 1168.0
average cost: 23.36
total execution time: 4.57
Average execution time: 0.09


In [86]:
print("GBFS-h2:")
total_cost = 0
total_execution_time = 0.0
total_no_solution = 0
for i in range(50):
    line_list = []
    input = open(str(i)+"_gbfs-h2_solution.txt","r")
    for lines in input:
        if lines == "No solution":
            total_execution_time += 60.0
            total_no_solution += 1
        else:
            line_list = input.readlines()
    if len(line_list) == 0:
        a = 0
    else:
        temp = line_list[len(line_list)-1]
        temp1 = temp.split()
        total_cost += float(temp1[0])
        total_execution_time += float(temp1[1])

        


print("total cost: " + str(total_cost))
print("average cost: " + str(round((total_cost/(50-total_no_solution)),2)))
print("total execution time: " + str(round(total_execution_time,2)))
print("Average execution time: " + str(round(total_execution_time/50,2)))

GBFS-h2:
total cost: 1202.0
average cost: 24.04
total execution time: 3.65
Average execution time: 0.07


In [88]:
print("astar-h1:")
total_cost = 0
total_execution_time = 0.0
total_no_solution = 0
for i in range(50):
    line_list = []
    input = open(str(i)+"_astar-h1_solution.txt","r")
    for lines in input:
        if lines == "No solution":
            total_execution_time += 60.0
            total_no_solution += 1
        else:
            line_list = input.readlines()
    if len(line_list) == 0:
        a = 0
    else:
        temp = line_list[len(line_list)-1]
        temp1 = temp.split()
        total_cost += float(temp1[0])
        total_execution_time += float(temp1[1])

        


print("total cost: " + str(total_cost))
print("average cost: " + str(round((total_cost/(50-total_no_solution)),2)))
print("total execution time: " + str(round(total_execution_time,2)))
print("Average execution time: " + str(round(total_execution_time/50,2)))

astar-h1:
total cost: 684.0
average cost: 13.68
total execution time: 56.82
Average execution time: 1.14


In [87]:
print("astar-h2:")
total_cost = 0
total_execution_time = 0.0
total_no_solution = 0
for i in range(50):
    line_list = []
    input = open(str(i)+"_astar-h2_solution.txt","r")
    for lines in input:
        if lines == "No solution":
            total_execution_time += 60.0
            total_no_solution += 1
        else:
            line_list = input.readlines()
    if len(line_list) == 0:
        a = 0
    else:
        temp = line_list[len(line_list)-1]
        temp1 = temp.split()
        total_cost += float(temp1[0])
        total_execution_time += float(temp1[1])

        


print("total cost: " + str(total_cost))
print("average cost: " + str(round((total_cost/(50-total_no_solution)),2)))
print("total execution time: " + str(round(total_execution_time,2)))
print("Average execution time: " + str(round(total_execution_time/50,2)))

astar-h2:
total cost: 683.0
average cost: 13.66
total execution time: 287.01
Average execution time: 5.74
